# Timer

In [6]:
from time import sleep
from datetime import datetime as dt

start_time = "10.12.2022, 22:25"
start_time = dt.strptime(start_time, "%d.%m.%Y, %H:%M")

while True:
    if start_time < dt.now():
        break
    else:
        sleep(10)

# Parameter optimization

In [7]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['PriceChange', 'LinearReg']
work_timeframe = '15m'
higher_timeframe = '1h'
opt_limit = 100
load = False

print(f'Timeframe is {work_timeframe}/{higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'PriceChange': {"low_price_quantile": [1 * (i +  1) for i in range(20)]}, 
                'LinearReg': {'timeperiod': [4, 6, 8, 10, 12], 'low_bound': [0]}
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Timeframe is 15m/1h, trade type is sell
Number of combinations is 100


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [03:39<00:00,  2.20s/it]


# Check local statistics

In [8]:
# 15m/1h
stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('e_ratio_rank', ascending=False).head(20)

,pattern,PriceChange_low_price_quantile,LinearReg_timeperiod,LinearReg_low_bound,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank
60,PriceChange_LinearReg,13,4,0,1.9924,2.4865,2.5177,2.5164,1.9409,1.9095,1.9322,1.8867,1.9320,1.9937,2.0631,2.0184,2.0175,2.0868,2.1670,2.0683,1.7576,1.4594,1.4230,1.3000,1.2395,1.1813,1.1911,1.1797,-1.39,-1.88,-2.37,-2.77,-2.39,-2.63,-2.70,-2.19,-2.33,-2.52,-2.91,-2.58,-3.22,-3.62,-3.45,-3.10,-2.59,-2.71,-2.66,-2.95,-3.42,-2.67,-2.52,-2.62,40,1.844196,-2.674583,33.767833,-106.983333
90,PriceChange_LinearReg,19,4,0,2.2294,2.6996,2.2884,2.0641,1.7068,1.7184,1.6673,1.6029,1.6003,1.4851,1.5145,1.5004,1.5057,1.5579,1.6127,1.5801,1.4356,1.2793,1.2617,1.1929,1.1585,1.1254,1.1409,1.1442,-1.14,-1.63,-2.01,-2.34,-2.12,-2.26,-2.09,-2.07,-2.03,-2.02,-2.25,-2.28,-2.86,-2.80,-2.84,-2.74,-2.16,-2.11,-2.12,-2.64,-2.41,-2.50,-2.41,-2.56,53,1.586338,-2.266250,31.075888,-120.111250
85,PriceChange_LinearReg,18,4,0,2.1858,2.6373,2.2448,2.0304,1.6798,1.6935,1.6443,1.6067,1.6040,1.4867,1.5165,1.4987,1.5042,1.5571,1.6127,1.5796,1.4334,1.2758,1.2580,1.1888,1.1543,1.1211,1.1368,1.1388,-1.14,-1.64,-2.02,-2.36,-2.16,-2.27,-2.12,-2.07,-2.09,-2.15,-2.34,-2.35,-2.92,-2.83,-2.85,-2.76,-2.25,-2.13,-2.35,-2.69,-2.69,-2.51,-2.44,-2.56,52,1.574546,-2.320417,29.876383,-120.661667
95,PriceChange_LinearReg,20,4,0,2.1252,2.5843,2.2256,2.0071,1.6628,1.6773,1.6310,1.5707,1.5479,1.4312,1.4601,1.4485,1.4551,1.5051,1.5322,1.5038,1.3738,1.2313,1.2186,1.1536,1.1174,1.0873,1.1021,1.1058,-1.13,-1.44,-1.99,-2.12,-1.92,-2.25,-1.93,-1.98,-1.95,-1.96,-2.16,-2.16,-2.35,-2.43,-2.83,-2.39,-2.15,-2.04,-2.01,-1.92,-2.35,-1.96,-1.94,-2.13,55,1.531575,-2.062083,29.236625,-113.414583
61,PriceChange_LinearReg,13,6,0,1.6796,2.1519,2.0752,2.1325,2.0406,2.0582,2.0812,2.0657,2.1157,2.1644,2.1957,2.1833,2.1994,2.2795,2.3816,2.3635,2.1915,1.6261,1.5534,1.3644,1.2940,1.2996,1.3139,1.3084,-1.34,-1.84,-2.27,-2.67,-2.29,-2.48,-2.16,-2.16,-2.41,-2.50,-2.44,-2.43,-3.02,-3.54,-3.20,-3.20,-2.78,-3.26,-2.69,-2.91,-3.01,-2.84,-2.56,-2.67,31,1.921637,-2.611250,28.570762,-80.948750
75,PriceChange_LinearReg,16,4,0,2.0998,2.5258,2.1101,2.0517,1.6498,1.6627,1.6870,1.6324,1.6286,1.4921,1.5258,1.4895,1.4492,1.4979,1.5495,1.5115,1.3574,1.1922,1.1727,1.1039,1.0678,1.0356,1.0524,1.0561,-1.32,-1.78,-2.19,-2.53,-2.24,-2.36,-2.16,-2.12,-2.16,-2.35,-2.43,-2.35,-2.99,-2.87,-2.85,-2.76,-2.25,-2.07,-2.06,-2.33,-2.38,-2.23,-2.44,-2.62,48,1.525062,-2.326667,25.203000,-111.680000
51,PriceChange_LinearReg,11,6,0,1.7567,2.5285,2.3430,2.3829,2.3191,2.2781,2.2584,2.2070,2.2460,2.2732,2.3046,2.2798,2.2955,2.3836,2.4618,2.4222,2.1863,1.5329,1.4569,1.2549,1.1837,1.1893,1.1972,1.1903,-1.35,-1.84,-2.27,-2.87,-2.29,-2.48,-2.16,-2.16,-2.25,-2.43,-2.43,-2.43,-2.35,-2.80,-3.20,-3.20,-2.78,-2.16,-2.62,-2.74,-2.35,-2.10,-2.48,-2.56,25,1.997162,-2.429167,24.929063,-60.729167
80,PriceChange_LinearReg,17,4,0,2.1011,2.5608,2.1440,1.9199,1.5890,1.6057,1.5552,1.5136,1.5132,1.4043,1.4357,1.4208,1.4287,1.4822,1.5291,1.4955,1.3570,1.2074,1.1914,1.1267,1.0952,1.0640,1.0798,1.0828,-1.23,-1.71,-2.10,-2.39,-2.16,-2.27,-2.12,-2.07,-2.09,-2.15,-2.34,-2.35,-2.99,-2.87,-2.85,-2.76,-2.25,-2.13,-2.35,-2.69,-2.69,-2.51,-2.44,-2.56,50,1.495962,-2.336250,24.798125,-116.812500
55,PriceChange_LinearReg,12,4,0,1.8608,2.2960,2.2611,2.3012,1.8070,1.7

In [5]:
# 15m/4h
stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('e_ratio_rank', ascending=False).head(20)

,pattern,PriceChange_low_price_quantile,LinearReg_timeperiod,LinearReg_low_bound,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank
14,PriceChange_LinearReg,3,12,0,2.1507,9.1589,9.1589,9.1589,10.0000,10.0000,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,-0.90,-3.32,-2.17,-1.93,-3.79,-3.27,-2.63,-2.31,-2.94,-2.60,-2.31,-2.14,-2.36,-1.93,-2.28,-2.82,-2.20,-1.80,-1.88,-2.29,-2.21,-2.40,-2.06,-1.97,2,9.198358,-2.354583,16.396717,-4.709167
13,PriceChange_LinearReg,3,10,0,2.1507,9.1589,9.1589,9.1589,10.0000,10.0000,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,-0.90,-3.32,-2.17,-1.93,-3.79,-3.27,-2.63,-2.31,-2.94,-2.60,-2.31,-2.14,-2.36,-1.93,-2.28,-2.82,-2.20,-1.80,-1.88,-2.29,-2.21,-2.40,-2.06,-1.97,2,9.198358,-2.354583,16.396717,-4.709167
50,PriceChange_LinearReg,11,4,0,1.3227,2.4615,2.6115,2.9298,2.8655,3.1707,3.0831,2.7546,2.7546,2.8093,2.8699,2.9520,3.0540,3.0945,3.0945,2.8997,2.3034,1.2110,1.1115,0.8560,0.7835,0.7767,0.7026,0.6396,-0.54,-0.82,-1.86,-1.67,-1.54,-1.62,-1.54,-1.54,-1.57,-1.62,-1.73,-1.35,-1.77,-1.89,-1.68,-1.26,-1.84,-1.59,-1.35,-0.93,-0.81,-0.48,-0.49,-0.39,11,2.213008,-1.328333,13.343092,-14.611667
55,PriceChange_LinearReg,12,4,0,1.3227,2.4615,2.6115,2.9298,2.8655,3.1707,3.0831,2.7546,2.7546,2.8093,2.8699,2.9520,3.0540,3.0945,3.0945,2.8997,2.3034,1.2110,1.1115,0.8560,0.7835,0.7767,0.7026,0.6396,-0.54,-0.82,-1.86,-1.67,-1.54,-1.62,-1.54,-1.54,-1.57,-1.62,-1.73,-1.35,-1.77,-1.89,-1.68,-1.26,-1.84,-1.59,-1.35,-0.93,-0.81,-0.48,-0.49,-0.39,11,2.213008,-1.328333,13.343092,-14.611667
56,PriceChange_LinearReg,12,6,0,1.3048,2.5071,2.6656,2.7911,2.7245,3.0561,2.9664,2.6273,2.6273,2.6860,2.7511,2.8027,2.8027,2.8461,2.8461,2.6548,2.0560,1.0555,0.9691,0.7430,0.6816,0.6756,0.6102,0.5547,-0.53,-1.33,-1.88,-1.46,-1.43,-1.62,-1.54,-1.42,-1.50,-1.41,-1.56,-1.20,-1.56,-1.81,-1.65,-1.17,-1.73,-1.26,-1.19,-0.88,-0.76,-0.41,0.16,-0.33,10,2.083558,-1.227917,10.835583,-12.279167
51,PriceChange_LinearReg,11,6,0,1.3048,2.5071,2.6656,2.7911,2.7245,3.0561,2.9664,2.6273,2.6273,2.6860,2.7511,2.8027,2.8027,2.8461,2.8461,2.6548,2.0560,1.0555,0.9691,0.7430,0.6816,0.6756,0.6102,0.5547,-0.53,-1.33,-1.88,-1.46,-1.43,-1.62,-1.54,-1.42,-1.50,-1.41,-1.56,-1.20,-1.56,-1.81,-1.65,-1.17,-1.73,-1.26,-1.19,-0.88,-0.76,-0.41,0.16,-0.33,10,2.083558,-1.227917,10.835583,-12.279167
29,PriceChange_LinearReg,6,12,0,0.9902,3.7682,4.4289,4.4289,5.1201,5.1201,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,3.9411,1.2719,-0.21,-1.97,-2.38,-1.45,-1.31,-1.48,-1.31,-1.38,-1.21,-1.38,-1.24,-1.48,-1.48,-1.45,-1.45,-1.45,-1.45,-1.31,-1.24,-1.17,-1.24,-1.31,-1.10,-1.31,3,4.437625,-1.365000,10.312875,-4.095000
18,PriceChange_LinearReg,4,10,0,0.9902,3.7682,4.4289,4.4289,5.1201,5.1201,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,3.9411,1.2719,-0.21,-1.97,-2.38,-1.45,-1.31,-1.48,-1.31,-1.38,-1.21,-1.38,-1.24,-1.48,-1.48,-1.45,-1.45,-1.45,-1.45,-1.31,-1.24,-1.17,-1.24,-1.31,-1.10,-1.31,3,4.437625,-1.365000,10.312875,-4.095000
24,PriceChange_LinearReg,5,12,0,0.9902,3.7682,4.4289,4.4289,5.1201,5.1201,4.8396,4

# Save new config data to config file

In [9]:
from config_updater import ConfigUpdater

ttype = 'sell'
pattern = ['PriceChange', 'LinearReg']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'PriceChange': {"low_price_quantile": [20]}, 
              'LinearReg': {'timeperiod': [4], 'low_bound': [0]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)

# Check global statistics

In [3]:
from glob import glob

ttype = 'sell'
pattern = ['PriceChange']
work_timeframe = '5m'
higher_timeframe = '1h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['PriceChange_low_price_quantile']).agg({'pct_right_forecast_avg': 'mean',
                                                                   'pct_price_diff_avg': 'mean',
                                                                   'forecast_rank': 'mean', 
                                                                   'price_rank': 'mean', 
                                                                   'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                                        ascending=False)
total_stat.head(20)

,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank,forecasts_num
PriceChange_low_price_quantile,,,,,
20,72.930243,-1.306944,-879.743681,-157.711944,3071
19,73.131424,-1.325521,-881.025174,-153.987604,3038
17,73.016458,-1.380833,-906.773576,-147.793403,2972
18,72.718576,-1.329062,-908.998681,-150.019097,3005
16,73.201181,-1.421007,-909.151806,-143.919722,2933
15,73.250174,-1.485451,-924.197604,-142.200174,2911
12,74.196285,-1.676250,-927.744236,-130.194201,2805
13,73.203299,-1.593021,-954.058576,-131.837604,2836
11,73.823993,-1.782292,-954.826875,-126.921111,2770
